In [1]:
using LinearAlgebra
using Plots
using GaussQuadrature
using SparseArrays
using DataFrames
using Printf

## Base

In [2]:
function ϕ(a::Int64)::Function
    if a == 1
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0-ξ₁)*(1.0-ξ₂)/4.0
        
    elseif a == 2
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0+ξ₁)*(1.0-ξ₂)/4.0
        
    elseif a == 3
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0+ξ₁)*(1.0+ξ₂)/4.0
        
    elseif a == 4
        return (ξ₁::Float64, ξ₂::Float64) -> (1.0-ξ₁)*(1.0+ξ₂)/4.0
        
    end
end

function ∂ϕ(variable::Int64, a)::Function
    if variable == 1
        if a == 1
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0-ξ₂)/4.0
        elseif a == 2
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0-ξ₂)/4.0
        elseif a == 3
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0+ξ₂)/4.0
        elseif a == 4
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0+ξ₂)/4.0
        end
        
    elseif variable == 2
        if a == 1
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0-ξ₁)/4.0
        elseif a == 2
            return (ξ₁::Float64, ξ₂::Float64) -> -(1.0+ξ₁)/4.0
        elseif a == 3
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0+ξ₁)/4.0
        elseif a == 4
            return (ξ₁::Float64, ξ₂::Float64) ->  (1.0-ξ₁)/4.0
        end
        
    end
end

∂ϕ (generic function with 1 method)

In [10]:
function quadratura_phis_K_2D(f1::Function, f2::Function, P::Vector{Float64}, W::Vector{Float64})::Float64
    quadratura = 0
    
    for (ξ₁, w₁) in zip(P, W)
        for (ξ₂, w₂) in zip(P, W)
            quadratura += (w₁*w₂) * (f1(ξ₁, ξ₂) * f2(ξ₁, ξ₂))
            #@printf("%f | %f | %f | %f \n", ξ₁, ξ₂, w₁, w₂)
        end
    end
    
    return quadratura
end

quadratura_phis_K_2D (generic function with 1 method)

In [4]:
function monta_K_local_2D(α::Float64, β::Float64, h₁::Float64, h₂::Float64, P::Vector{Float64}, W::Vector{Float64})::Matrix{Float64}
    K_local = zeros(4, 4)
    
    for a in 1:4
        for b in 1:4
            K_local[a, b] =   (α*h₂/h₁  ) * quadratura_phis_K_2D(∂ϕ(1,b), ∂ϕ(1,a), P, W)
                            + (α*h₁/h₂  ) * quadratura_phis_K_2D(∂ϕ(2,b), ∂ϕ(2,a), P, W)
                            + (β*h₁*h₂/4) * quadratura_phis_K_2D( ϕ(  b),  ϕ(  a), P, W)
        end
    end
    
    return K_local
end

monta_K_local_2D (generic function with 1 method)

In [11]:
α = 6.0
β = 0.0
h₁ = 1/4
h₂ = 1/4
P, W = legendre(2)

#quadratura_phis_K_2D( ϕ(  1),  ϕ(  1), P, W)
display(monta_K_local_2D(α, β, h₁, h₂, P, W))

4×4 Matrix{Float64}:
  2.0  -2.0  -1.0   1.0
 -2.0   2.0   1.0  -1.0
 -1.0   1.0   2.0  -2.0
  1.0  -1.0  -2.0   2.0